In [14]:
using Pkg   
using Distributions,Plots,Parameters,LinearAlgebra,Optim,Roots
@with_kw struct model             
    ## Model parameters:
    β = 0.95 # discount factor
    σ = 2 # RRA/IES
    θ = 0.5 # probability of no information update
    w = 1.0 # fixed riskless income
   
    ## aggregate state:
    nz = 2 # number of agg states
    γgrid = [0.98,1.02] # grid for aggregate state
    p_bg = 0.2 # probability of bad -> good
    p_gg = 0.6 # probability of good -> good
    γprob = [1-p_bg p_bg ; 1-p_gg p_gg] # transition matrix for aggregate state, [x,y] gives probability of transition from x to y

    ## asset grid:
    na = 100 # number of asset grid points
    amin = 0.0 # borrowing constraint
    amax = 1.0 # maximum asset level
    agrid = range(amin,stop=amax,length=na) # asset grid

    ## periods since update grid:
    kmin = 0 # minimum periods since update
    kmax = 20 # θ^kmax should be very small
    nk = 20 # number of periods since update grid points
    kgrid = range(kmin,stop=kmax,length=nk+1) # periods since update grid

    ## collect all
    stateind = collect(Iterators.product(1:na, 1:nz, 1:nz, 1:nk)) # matrix of state indicies (assets, true z, last observed z, nk)

    ## build a k period forward transition matrix
    γprob_kfwd = [ γprob^kgrid[ik+1] for ik in 1:nk]

    ## numerical parameters
    maxit = 20
    tol = 1e-5
end

# utility function
function u(par::model,c)
    @unpack σ = par
    if c <= 0
        return -9999999999
    end
    if σ == 1
        return log(c)
    else
        return c^(1-σ)/(1-σ)
    end
end;

include("VFI.jl");
include("aggregates.jl");

In [15]:
# guess pz's_
pzguess = [1.0,1.0];

In [16]:
V,apol = VFI(model(),pzguess);

([-3.3386354406315046 0.0; -3.307502326191787 0.0; … ; -1.9118189126928855 0.0; -1.9066653361107373 0.0;;; 0.0 -3.3386354406315046; 0.0 -3.3068835761531155; … ; 0.0 -1.9017158615714467; 0.0 -1.8966643360107274;;;; -3.122758604635805 0.0; -3.093475838439367 0.0; … ; -1.767920402644156 0.0; -1.7627668260620082 0.0;;; 0.0 -3.122758604635805; 0.0 -3.092886903246688; … ; 0.0 -1.7578173515227173; 0.0 -1.752765825961998;;;; -3.038318403721172 0.0; -3.0097834954945824 0.0; … ; -1.7127739237664947 0.0; -1.707620347184347 0.0;;; 0.0 -3.038318403721172; 0.0 -3.0092086660229707; … ; 0.0 -1.7026708726450561; 0.0 -1.6976193470843368;;;; … ;;;; -2.9825547775403596 0.0; -2.954531683460363 0.0; … ; -1.6767602789542426 0.0; -1.6716067023720949 0.0;;; 0.0 -2.9825547775403596; 0.0 -2.953970458938752; … ; 0.0 -1.666657227832804; 0.0 -1.6616057022720847;;;; -2.982554741980126 0.0; -2.9545316481881163 0.0; … ; -1.6767602560366082 0.0; -1.67160667945446 0.0;;; 0.0 -2.982554741980126; 0.0 -2.953970423666505; …

In [20]:
# test the distribution transition fct:
jd_1 = jdguess(model());
jd_2 = jdtrans(model(),jd_1,apol,1); # 1 here = bad state today